In [ ]:
# 以下所有代码只能在聚宽的研究环境下运行
import datetime
import json
import traceback
import numpy as np
import pandas as pd
from jqdata import opt
from pathlib import Path
from datetime import datetime
from time import sleep

In [ ]:
def get_mission_list(count: int, num_per_group: int):
    group_num, remind = divmod(count, num_per_group)
    body = np.arange(group_num * num_per_group).reshape((group_num, num_per_group)).tolist()
    tail = np.arange(group_num * num_per_group, count).tolist()
    if tail:
        body.append(tail)
    return body


def get_all_option_basic():
    option_q = query(opt.OPT_CONTRACT_INFO)
    return opt.run_query(option_q) 


def get_option_basic(underlying_symbol: str = '510050.XSHG'):
    option_q = query(opt.OPT_CONTRACT_INFO).filter(opt.OPT_CONTRACT_INFO.underlying_symbol==underlying_symbol)
    return opt.run_query(option_q)
        
        
def is_file_exist(filename: str):
    filepath = Path.cwd().joinpath(filename)
    return filepath.exists()


def get_bar_by_code(code: str):
    price_q = query(opt.OPT_DAILY_PRICE).filter(opt.OPT_DAILY_PRICE.code==code)
    return opt.run_query(price_q)
    

def get_bar_by_date(date: datetime.date):
    price_q = query(opt.OPT_DAILY_PRICE).filter(opt.OPT_DAILY_PRICE.date==date)
    first_df = opt.run_query(price_q)
    last_id = first_df.iloc[-1]['id']
    n = len(first_df)
    
    df_list = []
    df_list.append(first_df)
    
    while n == 4000:
        q = query(opt.OPT_DAILY_PRICE).filter(opt.OPT_DAILY_PRICE.date==date, opt.OPT_DAILY_PRICE.id > last_id)
        df = opt.run_query(q)
        n = len(df)
        if not n:
            break
        else:
            df_list.append(df)
            sleep(1)
            print('休息1s')
            
    all_df = pd.concat(df_list, axis=0)
    print(f'日价格{date}合成完成, 数据量：{len(all_df)}')
    return all_df
    

def get_bar_in_sub_list(option_basic: pd.DataFrame, sub_mission_list: list):
    df_list = []
    is_all_delist = True
    for mission_id in sub_mission_list:
        contract = option_basic.iloc[mission_id]
        code = contract['code']
        name = contract['name']
        trading_code = contract['trading_code']
        
        df = get_bar_by_code(code)
        df['trading_code'] = trading_code
        df['name'] = name
        status = contract['contract_status']
        df['status'] = status
        if status == 'LIST':
            is_all_delist = False
        df_list.append(df)
#         print(f"{code}-{trading_code}-{name}数据获取成功")
    group_df = pd.concat(df_list, axis=0)
    return group_df, is_all_delist
        
    
def get_expire_dates(underlying_symbol: str):
    df = get_option_basic(underlying_symbol)
    expire_dates = df.expire_date.drop_duplicates()
    expire_dates = expire_dates[expire_dates < datetime.now().date()]
    expire_dates.sort_values(inplace=True)
    expire_dates.reset_index(drop=True, inplace=True)
    return expire_dates


def dt_to_str(dt: datetime):
    return dt.strftime('%Y-%m-%d')


def get_last_day_price(underlying_symbol: str, date: datetime.date):
    price = get_bar_by_date(date)
    
    price['underlying'] = price['code'].map(basic_db['underlying_symbol'])
    price.dropna(inplace=True)
    price['expire_date'] = price['code'].map(basic_db['expire_date'])
    price = price[price['expire_date'] == date]

    price['name'] = price['code'].map(basic_db['name'])
    price['type'] = price['code'].map(basic_db['contract_type'])
    price['strike'] = price['code'].map(basic_db['exercise_price'])
    price['underlying_close'] = underlying_price.loc[date]['close']

    price.loc[price['type'] == 'CO', 'intrinsic'] = price['underlying_close'] - price['strike']
    price.loc[price['type'] == 'PO', 'intrinsic'] = price['strike'] - price['underlying_close']
    price.loc[price['intrinsic'] < 0, 'intrinsic'] = 0
    return price

def get_all_last_day_price(underlying_symbol: str):
    df_list = []
    
    dates = get_expire_dates(underlying_symbol)
    for date in dates:
        df = get_last_day_price(underlying_symbol, date)
        df_list.append(df)
        
    all_df = pd.concat(df_list, axis=0)
    return all_df
    
    
basic_db = None
underlying_price = None

In [ ]:
# 下载已归档数据

num_per_group = 100
option_basic = get_option_basic()
all_mission_list = get_mission_list(len(option_basic), num_per_group)

for idx, sub_mission_list in enumerate(all_mission_list):
    filename = "option_daily_bar_archived_{:0>4d}.csv".format(idx)
    if not is_file_exist(filename):
        try:
            df, is_all_delist = get_bar_in_sub_list(option_basic, sub_mission_list)
        except:
            print(f"获取单组数据出错触发异常：\n{traceback.format_exc()}")
        else:
            if len(sub_mission_list) == num_per_group and is_all_delist:
                df.to_csv(filename)
            else:
                filename = filename = "option_daily_bar_unarchived_{:0>4d}.csv".format(idx)
                df.to_csv(filename)
            print(f"任务ID系列：{sub_mission_list}")
            print("单组数据保存成功！")
            print("=" * 50)
    else:
        print(f"{filename}已归档无需下载")
        pass
    

In [ ]:
# 获取某个标的期权最后到期日的价格数据

underlying_symbol = '510050.XSHG'

basic_db = get_option_basic(underlying_symbol).set_index('code')
underlying_price = get_price(underlying_symbol, start_date=datetime(2015, 2, 9), end_date=datetime.now(), fq=None)

df = get_all_last_day_price(underlying_symbol)
df.to_csv(f'{underlying_symbol}.last_day.price.csv')